In [1]:
!pip install galois

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 52.2 MB/s eta 0:00:00


**Класс HammingCode**

In [2]:
import numpy as np
import galois
from itertools import product


class HammingCode:
    def __init__(self, q, m):
        """
        Конструктор класса.

        Параметры:
        q — порядок конечного поля GF(q)
        m — число проверочных символов

        Инициализируются параметры кода (n, k), строятся матрицы H и G,
        формируется таблица синдромов для быстрого декодирования.
        """
        self.q = q
        self.m = m
        self.GF = galois.GF(q)

        self.n = (q**m - 1) // (q - 1)
        self.k = self.n - m

        self.H = self._build_H()
        self.G = self._build_G_from_kernel(self.H)
        self._build_syndrome_table()

    # ------------------------------------------------------------
    def _build_H(self):
        """
        Построение проверочной матрицы H размера m × n.

        - генерирует все ненулевые векторы GF(q)^m,
        - нормирует каждый вектор так, чтобы первый ненулевой элемент стал равен 1,
        - исключает пропорциональные векторы,
        - формирует столбцы H из представителей одномерных подпространств.

        Выход: H — матрица в поле GF(q)
        """
        GF = self.GF
        m = self.m

        reps = []
        seen = set()

        for vec in product(range(self.q), repeat=m):
            if all(v == 0 for v in vec):
                continue

            v = GF(list(vec))

            for i, a in enumerate(v):
                if a != 0:
                    v_norm = v * (GF(1) / a)
                    key = tuple(int(x) for x in v_norm)
                    break

            if key not in seen:
                seen.add(key)
                reps.append(key)

        reps.sort()
        return GF(np.array(reps).T)

    # ------------------------------------------------------------
    def _build_G_from_kernel(self, H):
        """
        Построение порождающей матрицы G как базиса ядра H.

        - выполняет приведение H к ступенчатому виду над GF(q),
        - определяет свободные столбцы,
        - формирует базисные векторы пространства решений Hx = 0,
        - собирает из них матрицу G размера k × n.

        Вход: H — проверочная матрица

        Выход: G — порождающая матрица
        """
        GF = self.GF
        H = GF(H.copy())

        m, n = H.shape
        A = H.copy()

        pivot_rows = []
        pivot_cols = []

        row = 0
        for col in range(n):
            pivot = None
            for r in range(row, m):
                if A[r, col] != 0:
                    pivot = r
                    break
            if pivot is None:
                continue

            if pivot != row:
                A[[row, pivot]] = A[[pivot, row]]

            inv = GF(1) / A[row, col]
            A[row] *= inv

            for r in range(m):
                if r != row and A[r, col] != 0:
                    A[r] -= A[r, col] * A[row]

            pivot_rows.append(row)
            pivot_cols.append(col)
            row += 1
            if row == m:
                break

        free_cols = [j for j in range(n) if j not in pivot_cols]

        basis = []
        for free in free_cols:
            v = GF.Zeros(n)
            v[free] = GF(1)
            for pr, pc in zip(pivot_rows, pivot_cols):
                v[pc] = GF(0) - A[pr, free]
            basis.append(v)

        return GF(np.vstack(basis))

    # ------------------------------------------------------------
    def _build_syndrome_table(self):
        """
        Построение таблицы синдромов для быстрого декодирования.

        - перебирает все одноошибочные векторы e: ошибка в одной позиции, значение ∈ GF(q)\{0},
        - вычисляет их синдромы s = H * eᵀ,
        - сохраняет соответствие: синдром → вектор ошибки.

        Назначение: ускоряет декодирование: поиск ошибки становится операцией словаря.

        Выход: self.syndrome_table — словарь {синдром → ошибка}
        """
        GF = self.GF
        m, n = self.H.shape

        self.syndrome_table = {}

        zero = tuple(int(x) for x in GF.Zeros(m))
        self.syndrome_table[zero] = GF.Zeros(n)

        for pos in range(n):
            for a in range(1, self.q):
                e = GF.Zeros(n)
                e[pos] = GF(a)
                S = (self.H @ e.reshape(-1, 1)).reshape(-1)
                key = tuple(int(x) for x in S)
                if key not in self.syndrome_table:
                    self.syndrome_table[key] = e

    # ------------------------------------------------------------
    def encode(self, msg):
        """
        Кодирование информационного слова.

        - преобразует информационный вектор u длины k в кодовое слово c длины n,
          используя матрицу G:  c = uG.

        Вход: msg — список или массив длины k над GF(q)

        Выход: кодовое слово длины n
        """
        GF = self.GF
        msg = GF(msg).reshape(-1)
        if msg.size != self.k:
            raise ValueError(f"Неверная длина сообщения, ожидается {self.k}")
        return msg @ self.G

    # ------------------------------------------------------------
    def decode(self, received):
        """
        Синдромное декодирование с исправлением одной ошибки.

        - вычисляет синдром s = H rᵀ,
        - по таблице синдромов определяет позицию и величину ошибки,
        - исправляет ошибку: r - e.

        Вход: received — принятый вектор длины n

        Выход: исправленный вектор длины n
        """
        GF = self.GF
        r = GF(received).reshape(-1)
        S = (self.H @ r.reshape(-1, 1)).reshape(-1)
        key = tuple(int(x) for x in S)

        e = self.syndrome_table.get(key)
        return r if e is None else r - e

    # ------------------------------------------------------------
    def decode_to_message(self, received):
        """
        Восстановление исходного сообщения.

        - сначала исправляет ошибки (decode),
        - решает систему линейных уравнений Gᵀ mᵀ = cᵀ,
        - выделяет информационную часть вектора.

        Вход: received — принятый вектор длины n

        Выход: информационное слово длины k
        """
        GF = self.GF
        c = self.decode(received).reshape(-1)

        G = self.G
        k, n = G.shape

        A = GF(G.T.copy())
        b = GF(c).reshape(-1, 1)

        Ab = GF(np.hstack([A, b]))

        rows, cols = Ab.shape
        num_vars = k

        r = 0
        pivot_cols = []

        for col in range(num_vars):
            pivot = None
            for i in range(r, rows):
                if Ab[i, col] != 0:
                    pivot = i
                    break
            if pivot is None:
                continue

            if pivot != r:
                Ab[[r, pivot]] = Ab[[pivot, r]]

            inv = GF(1) / Ab[r, col]
            Ab[r] *= inv

            for i in range(rows):
                if i != r and Ab[i, col] != 0:
                    Ab[i] -= Ab[i, col] * Ab[r]

            pivot_cols.append(col)
            r += 1
            if r == rows:
                break

        x = GF.Zeros(num_vars)
        for row_idx, col_idx in enumerate(pivot_cols):
            x[col_idx] = Ab[row_idx, -1]

        return x


<>:142: SyntaxWarning: invalid escape sequence '\{'
<>:142: SyntaxWarning: invalid escape sequence '\{'
/tmp/ipython-input-572394361.py:142: SyntaxWarning: invalid escape sequence '\{'
  - перебирает все одноошибочные векторы e: ошибка в одной позиции, значение ∈ GF(q)\{0},


# Примеры


In [3]:
print("=== Примеры ===\n")

# ---------------------- GF(2), m=3 ----------------------
hc2 = HammingCode(2, 3)
msg2 = [1, 0, 1, 1]
code2 = hc2.encode(msg2)
r2 = code2.copy()
r2[3] += hc2.GF(1)

print("GF(2), m=3")
print("Сообщение:              ", msg2)
print("Кодовое слово:          ", np.array(code2, dtype=int))
print("Принято с ошибкой:      ", np.array(r2, dtype=int))
print("Исправленное слово:     ", np.array(hc2.decode(r2), dtype=int))
print("Восстановленное сообщение:", np.array(hc2.decode_to_message(r2), dtype=int))
print("\n")


# ---------------------- GF(3), m=2 ----------------------
hc3 = HammingCode(3, 2)
msg3 = [1, 2]
code3 = hc3.encode(msg3)
r3 = code3.copy()
r3[1] += hc3.GF(2)

print("GF(3), m=2")
print("Сообщение:              ", msg3)
print("Кодовое слово:          ", np.array(code3, dtype=int))
print("Принято с ошибкой:      ", np.array(r3, dtype=int))
print("Исправленное слово:     ", np.array(hc3.decode(r3), dtype=int))
print("Восстановленное сообщение:", np.array(hc3.decode_to_message(r3), dtype=int))
print("\n")


# ---------------------- GF(5), m=2 ----------------------
hc5 = HammingCode(5, 2)
msg5 = [3, 4, 1, 0]
code5 = hc5.encode(msg5)
r5 = code5.copy()
r5[2] += hc5.GF(3)

print("GF(5), m=2")
print("Сообщение:              ", msg5)
print("Кодовое слово:          ", np.array(code5, dtype=int))
print("Принято с ошибкой:      ", np.array(r5, dtype=int))
print("Исправленное слово:     ", np.array(hc5.decode(r5), dtype=int))
print("Восстановленное сообщение:", np.array(hc5.decode_to_message(r5), dtype=int))


=== Примеры ===

GF(2), m=3
Сообщение:               [1, 0, 1, 1]
Кодовое слово:           [0 1 1 0 0 1 1]
Принято с ошибкой:       [0 1 1 1 0 1 1]
Исправленное слово:      [0 1 1 0 0 1 1]
Восстановленное сообщение: [1 0 1 1]


GF(3), m=2
Сообщение:               [1, 2]
Кодовое слово:           [1 0 1 2]
Принято с ошибкой:       [1 2 1 2]
Исправленное слово:      [1 0 1 2]
Восстановленное сообщение: [1 2]


GF(5), m=2
Сообщение:               [3, 4, 1, 0]
Кодовое слово:           [1 2 3 4 1 0]
Принято с ошибкой:       [1 2 1 4 1 0]
Исправленное слово:      [1 2 3 4 1 0]
Восстановленное сообщение: [3 4 1 0]
